In [1]:
%pylab inline
import sys; sys.path.insert(0, "../")
import numpy as np
import timeit
from emcee.autocorr import *
from ensemble_sampler import *

Populating the interactive namespace from numpy and matplotlib


### Example of sampling from Rosenbrock density  
$f(x_1, x_2) \propto \exp(-[a(x_2 - x_1^2)^2 + (1-x_1)^2]\ / \ b)$  

In [2]:
nwalkers = 250
batch_size = 50
niters = 100000

In [3]:
def run(proposal, batch_size=50, niters=1000, n=5, pre=0, title='', plot=False):
    sampler = Sampler(dim=2, t_dist=Rosenbrock(), proposal=proposal, nwalkers=nwalkers)

    acc_r = 0.0
    for i in range(n):
        sampler.reset()
        p0 = np.random.randn(2*nwalkers).reshape([nwalkers, 2])
        if pre > 0:
            s = Sampler(dim=2, t_dist=Rosenbrock(), proposal=PCNWalkMove(s=None, scale=0.2), nwalkers=nwalkers)
            p0 = s.run_mcmc(pre, batch_size=batch_size, p0=p0).curr_pos
        start = timeit.default_timer()
        hist = sampler.run_mcmc(niters-pre, batch_size=batch_size, p0=p0)
        print 'finishes loop %d in %.2f seconds' % (i, float(timeit.default_timer() - start))
        acc_r += float(100*hist.acceptance_rate.mean())
        print 'avg accept rate: %.2f%s' % (float(100*hist.acceptance_rate.mean()), '%')
        try:
            print 'auto-correlation time: %s' % sampler.auto_corr()
        except AutocorrError, err:
            print err
    if plot:
        hist.plot_scatter(dim=[[0, 1]])
        sns.plt.title(title)
    print 'avg_acc_r: %.2f%s' % (float(acc_r) / 5.0, '%')

##### Not using ensemble, use isotropic gaussian proposal with scale=0.2

In [4]:
proposal = PCNWalkMove(s=None, scale=0.2)
run(proposal, batch_size=batch_size, niters=niters, title='Non-ensemble')

finishes loop 0 in 53.77 seconds
avg accept rate: 47.70%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 1 in 51.40 seconds
avg accept rate: 46.33%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 2 in 55.28 seconds
avg accept rate: 46.42%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 3 in 52.44 seconds
avg accept rate: 46.70%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 4 in 51.72 seconds
avg accept rate: 46.84%
The chain is too short to reliably estimate the autocorrelation time
avg_acc_r: 46.80%


#### Use ensemble of size 3, scale=0.2.

In [5]:
proposal = PCNWalkMove(s=3, scale=0.2)
run(proposal, batch_size=batch_size, niters=niters, title='Ensemble of size 3', pre=1000)

finishes loop 0 in 94.41 seconds
avg accept rate: 20.59%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 1 in 95.04 seconds
avg accept rate: 19.85%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 2 in 96.03 seconds
avg accept rate: 19.37%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 3 in 96.76 seconds
avg accept rate: 20.05%
The chain is too short to reliably estimate the autocorrelation time
finishes loop 4 in 104.04 seconds
avg accept rate: 19.98%
The chain is too short to reliably estimate the autocorrelation time
avg_acc_r: 19.97%


#### Ensemble with pCN, beta=0.4   
This could come from the theory behind pCN sampling, i.e. it preserves the underlying Gaussian measure.

In [6]:
proposal = PCNWalkMove(s=3, beta=0.4, symmetric=True)
run(proposal, batch_size=batch_size, niters=niters, title='Ensemble with pCN')

finishes loop 0 in 102.94 seconds
avg accept rate: 87.81%
auto-correlation time: [ 51.33542751  29.45147128]
finishes loop 1 in 104.28 seconds
avg accept rate: 87.80%
auto-correlation time: [ 62.02899983  29.29339531]
finishes loop 2 in 100.57 seconds
avg accept rate: 87.82%
auto-correlation time: [ 50.39003079  25.32409016]
finishes loop 3 in 102.91 seconds
avg accept rate: 87.80%
auto-correlation time: [ 53.96154952  32.47967119]
finishes loop 4 in 105.64 seconds
avg accept rate: 87.82%
auto-correlation time: [ 60.38555997  26.10291383]
avg_acc_r: 87.81%


#### Ensemble with pCN, start from the end position of isotropic gaussian proposal

In [10]:
proposal=PCNWalkMove(s=3, beta=0.9)
run(proposal, batch_size=batch_size, niters=niters, pre=1000, title='pCN with ensemble, random walk for first 1000 steps')

finishes loop 0 in 104.27 seconds
avg accept rate: 83.51%
auto-correlation time: [  30.89257599  108.22763182]
finishes loop 1 in 102.26 seconds
avg accept rate: 83.52%
auto-correlation time: [  55.77165899  178.11962717]
finishes loop 2 in 115.85 seconds
avg accept rate: 83.47%
auto-correlation time: [  96.88611296  268.06988467]
finishes loop 3 in 102.26 seconds
avg accept rate: 83.49%
auto-correlation time: [ 104.40586397  356.10501769]
finishes loop 4 in 101.17 seconds
avg accept rate: 83.57%
auto-correlation time: [ 12.18775412  31.23693306]
avg_acc_r: 83.51%


#### pCN without ensemble, beta=0.4  
Check affine invariance of pCN. Notice this does not totally make sense.

In [8]:
proposal = PCNWalkMove(s=None, beta=0.4)
run(proposal, batch_size=batch_size, niters=niters, title='pCN without ensemble', pre=1000)

finishes loop 0 in 61.64 seconds
avg accept rate: 54.06%
auto-correlation time: [ 25.09599218  30.06170035]
finishes loop 1 in 54.55 seconds
avg accept rate: 54.04%
auto-correlation time: [ 29.06900346  28.7523777 ]
finishes loop 2 in 55.61 seconds
avg accept rate: 54.04%
auto-correlation time: [ 27.24763367  29.68079734]
finishes loop 3 in 51.64 seconds
avg accept rate: 54.05%
auto-correlation time: [ 28.38435496  31.60712574]
finishes loop 4 in 49.17 seconds
avg accept rate: 54.03%
auto-correlation time: [ 27.92874017  31.3003573 ]
avg_acc_r: 54.05%
